# Singular Value Decomposition

[Reference Repository: pytorch-lora by Umar Jamil](https://github.com/hkproj/pytorch-lora)

In [1]:
import torch
import numpy as np
_ = torch.manual_seed(0)

Generate a rank-deficient matrix W (40% parameters of the original model)

In [2]:
d, k = 10, 10

# This way we can generate a rank-deficient matrix
W_rank = 2
W = torch.randn(d,W_rank) @ torch.randn(W_rank,k)
print(W)

tensor([[-1.0797,  0.5545,  0.8058, -0.7140, -0.1518,  1.0773,  2.3690,  0.8486,
         -1.1825, -3.2632],
        [-0.3303,  0.2283,  0.4145, -0.1924, -0.0215,  0.3276,  0.7926,  0.2233,
         -0.3422, -0.9614],
        [-0.5256,  0.9864,  2.4447, -0.0290,  0.2305,  0.5000,  1.9831, -0.0311,
         -0.3369, -1.1376],
        [ 0.7900, -1.1336, -2.6746,  0.1988, -0.1982, -0.7634, -2.5763, -0.1696,
          0.6227,  1.9294],
        [ 0.1258,  0.1458,  0.5090,  0.1768,  0.1071, -0.1327, -0.0323, -0.2294,
          0.2079,  0.5128],
        [ 0.7697,  0.0050,  0.5725,  0.6870,  0.2783, -0.7818, -1.2253, -0.8533,
          0.9765,  2.5786],
        [ 1.4157, -0.7814, -1.2121,  0.9120,  0.1760, -1.4108, -3.1692, -1.0791,
          1.5325,  4.2447],
        [-0.0119,  0.6050,  1.7245,  0.2584,  0.2528, -0.0086,  0.7198, -0.3620,
          0.1865,  0.3410],
        [ 1.0485, -0.6394, -1.0715,  0.6485,  0.1046, -1.0427, -2.4174, -0.7615,
          1.1147,  3.1054],
        [ 0.9088,  

Evaluate the rank of the matrix W

In [3]:
W_rank = np.linalg.matrix_rank(W)
print(f'Rank of W: {W_rank}')

Rank of W: 2


Calculate the SVD decomposition of the W matrix.

In [4]:
# Perform SVD on W (W = UxSxV^T)
# SVD represent the scaling factors along each dimension
# U is the rotation toward the standard basis
# S is the scaling factor
# V is the rotation back to oriinal direction of the eigenvectors
# V is the transpose of U so the stardard basis roation can be rotated back the original direction of the eigenvectors
U, S, V = torch.svd(W)

# print(U, U.shape)
# print(S, S.shape)
# print(V, V.shape)
# For rank-r factorization, keep only the first r singular values (and corresponding columns of U and V)

# everything from the first dimension and rank elements from the second dimension
U_r = U[:, :W_rank]
# create a diagonal matrix with the first W scaling factors
S_r = torch.diag(S[:W_rank])
# everything from the first dimension and rank elements from the second dimension, then transposed
V_r = V[:, :W_rank].t()  # Transpose V_r to get the right dimensions

# Compute B = U_r * S_r and A = V_r
# (10, W_rank) @ (W_rank, W_rank) = (10, W_rank)
B = U_r @ S_r
# (W_rank, 10)
A = V_r
print(f'Shape of B: {B.shape}')
print(f'Shape of A: {A.shape}')

Shape of B: torch.Size([10, 2])
Shape of A: torch.Size([2, 10])


Given the same input, check the output using the original W matrix and the matrices resulting from the decomposition.

In [5]:
# Generate random bias and input
bias = torch.randn(d)
x = torch.randn(d)

# Compute y = Wx + bias
y = W @ x + bias
# Compute y' = (B*A)x + bias
y_prime = (B @ A) @ x + bias

print("Original y using W:\n", y)
print("")
print("y' computed using BA:\n", y_prime)

Original y using W:
 tensor([ 7.2684e+00,  2.3162e+00,  7.7151e+00, -1.0446e+01, -8.1639e-03,
        -3.7270e+00, -1.1146e+01,  2.0207e+00, -9.6258e+00, -4.1163e+00])

y' computed using BA:
 tensor([ 7.2684e+00,  2.3162e+00,  7.7151e+00, -1.0446e+01, -8.1638e-03,
        -3.7270e+00, -1.1146e+01,  2.0207e+00, -9.6258e+00, -4.1163e+00])


In [6]:
print("Total parameters of W: ", W.nelement())
print("Total parameters of B and A: ", B.nelement() + A.nelement())

Total parameters of W:  100
Total parameters of B and A:  40
